# Generative Adversarial Nets

The Generative Adversarial Nets (GAN) is a framework for estimating generative models via an adversarial process, in which we simultaneously train two models: a generative model G that captures the data distribution, and a discriminative model D that estimates the probability that a sample came from the training data rather than G. Both G and D could be a non-linear mapping function, such as a multi-layer perceptron neural network.


In [1]:
IMG_ROWS = 28
IMG_COLS = 28
CHANNELS = 1

IMG_SHAPE = (IMG_ROWS, IMG_COLS, CHANNELS)

LATENT_DIM = 100

NUM_CLASSES = 10

In [2]:
import os

for file in os.listdir('images'):
    os.remove(os.path.join('images', file))

####  Preparing the data

In order to prepare the data:
1. We will normalize the images and scale them between -1 and 1.
2. We will reshape the images from 28x28 to 28x28x1

In [3]:
from keras.datasets import mnist

import numpy as np

def prepare_data():
    # Load the dataset
    (X_train, _), (_, _) = mnist.load_data()

    # Rescale -1 to 1

    # Normalize data -> from 0 - 127 to -1 - 1
    X_train = X_train / 127.5 - 1.
    # Add channel dimension
    X_train = np.expand_dims(X_train, axis=3)

    return X_train
    

2024-01-24 14:01:18.678214: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-24 14:01:18.679811: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-24 14:01:18.706327: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-24 14:01:18.706874: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-24 14:01:19.255168: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

#### Creating the model

Model architecture:
0. `Input layer` (100 nodes: LATENT_DIM)
1. `Dense layer` (256 nodes) + `LeakyReLU activation` (alpha=0.2)
2. `BatchNormalization layer` (momentum of 0.8)
3. `Dense layer` (512 nodes) + `LeakyReLU activation` (alpha=0.2)
4. `BatchNormalization layer` (momentum of 0.8)
5. `Dense layer` (1024 nodes) + `LeakyReLU activation` (alpha=0.2)
6. `BatchNormalization layer` (momentum of 0.8)
7. `Dense layer` (28*28*1 nodes) + `Tanh activation`
8. `Reshape layer` (28x28x1: sum(IMG_SHAPE))

In [4]:
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.models import Sequential, Model

def build_generator():

    model = Sequential()

    model.add(Dense(256, input_dim=LATENT_DIM))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(IMG_SHAPE), activation='tanh'))
    model.add(Reshape(IMG_SHAPE))

    model.summary()

    noise = Input(shape=(LATENT_DIM,))
    img = model(noise)

    return Model(noise, img)

#### Creating the discriminator

Model architecture:
0. `Input layer` (IMG_SHAPE)
1. `Flatten layer`
2. `Dense layer` (512 nodes) + `LeakyReLU activation` (alpha=0.2)
3. `Dense layer` (256 nodes) + `LeakyReLU activation` (alpha=0.2)
4. `Dense layer` (1 node) + `Sigmoid activation`

In [5]:
def build_discriminator():

    model = Sequential()

    model.add(Flatten(input_shape=IMG_SHAPE))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    img = Input(shape=IMG_SHAPE)
    validity = model(img)

    return Model(img, validity)

#### Displaying the images

We will display the generated images every 10 epochs.

In [6]:
import matplotlib.pyplot as plt

def sample_images(epoch, generator):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, LATENT_DIM))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/%d.png" % epoch)
    plt.close()

#### Creating the GAN

OPtimizer: Adam (lr=0.0002, beta_1=0.5)

GAN Architecture:
1. Build the descriminator + compile it
2. Create the GAN model:
    1. Set the descriminator's weights to non-trainable
    2. Input layer (LATENT_DIM)
    3. Put Generator after input layer
    4. Put Discriminator after Generator
    5. Create the model with Input layer and Discriminator
    6. Compile the model

In [7]:
from keras.optimizers import Adam

OPTIMIZER = Adam(0.0002, 0.5)

def create_model():

    # Create the discriminator
    discriminator = build_discriminator()
    discriminator.compile(loss='binary_crossentropy',
        optimizer=OPTIMIZER,
        metrics=['accuracy'])
    
    # Create the generator
    generator = build_generator()

    # The generator takes noise as input and generates imgs
    z = Input(shape=(LATENT_DIM,))
    img = generator(z)

    # For the combined model we will only train the generator
    discriminator.trainable = False
    # The discriminator takes generated images as input and determines validity
    validity = discriminator(img)
    
    combined = Model(z, validity)
    # The combined model  (stacked generator and discriminator)
    # Trains the generator to fool the discriminator
    combined.compile(loss='binary_crossentropy', optimizer=OPTIMIZER)
    
    return discriminator, generator, combined
    


#### Training the GAN

In order to train the GAN:
1. Create a batch of random noise
2. Generate images from the noise
3. Create a batch of real images
4. Train the discriminator on the real images
5. Train the discriminator on the generated images
6. Create a batch of random noise
7. Train the GAN on the random noise


In [8]:

class GAN():
    def __init__(self):
        self.discriminator, self.generator, self.combined  = create_model()

    def train(self, epochs, batch_size=128, sample_interval=50):

        X_train = prepare_data()

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, LATENT_DIM))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid) # Learn to recognize real images
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake) # Learn to recognize fake images
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake) # Average the two losses

            noise = np.random.normal(0, 1, (batch_size, LATENT_DIM))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                sample_images(epoch, self.generator)

#### Generating images

Train the GAN for 2001 epochs, with a batch size of 32.

In [9]:
gan = GAN()
gan.train(epochs=2001, batch_size=32, sample_interval=200)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 257       
                                                                 
Total params: 533,505
Trainable params: 533,505
Non-trai

2024-01-24 14:01:20.864254: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [32,1]
	 [[{{node Placeholder/_1}}]]
2024-01-24 14:01:21.096870: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [32,1]
	 [[{{node Placeholder/_1}}]]
2024-01-24 14:01:21.104498: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [32,1]
	

0 [D loss: 1.070664, acc.: 39.06%] [G loss: 1.070570]
1/1 [==============================] - 0s 11ms/step
1 [D loss: 0.426000, acc.: 96.88%] [G loss: 1.085278]
1/1 [==============================] - 0s 11ms/step
2 [D loss: 0.327299, acc.: 90.62%] [G loss: 1.058955]
1/1 [==============================] - 0s 12ms/step
3 [D loss: 0.317092, acc.: 89.06%] [G loss: 1.110482]
1/1 [==============================] - 0s 12ms/step
4 [D loss: 0.279915, acc.: 93.75%] [G loss: 1.162942]


2024-01-24 14:01:21.926238: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [32,1]
	 [[{{node Placeholder/_1}}]]
2024-01-24 14:01:21.932219: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [32,1]
	 [[{{node Placeholder/_1}}]]
2024-01-24 14:01:21.937552: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [32,1]
	

1/1 [==============================] - 0s 15ms/step
5 [D loss: 0.240166, acc.: 100.00%] [G loss: 1.299427]
1/1 [==============================] - 0s 14ms/step
6 [D loss: 0.218865, acc.: 100.00%] [G loss: 1.386309]
1/1 [==============================] - 0s 15ms/step
7 [D loss: 0.173198, acc.: 100.00%] [G loss: 1.532442]
1/1 [==============================] - 0s 11ms/step
8 [D loss: 0.174471, acc.: 98.44%] [G loss: 1.680610]


...

2024-01-24 14:03:41.780464: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [32,1]
	 [[{{node Placeholder/_1}}]]
2024-01-24 14:03:41.795231: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [32,1]
	 [[{{node Placeholder/_1}}]]
2024-01-24 14:03:41.808875: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [32,1]
	

1/1 [==============================] - 0s 25ms/step
1996 [D loss: 0.560521, acc.: 68.75%] [G loss: 1.234920]
1/1 [==============================] - 0s 20ms/step
1997 [D loss: 0.556238, acc.: 73.44%] [G loss: 1.081208]
1/1 [==============================] - 0s 19ms/step


2024-01-24 14:03:41.999808: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [32,1]
	 [[{{node Placeholder/_1}}]]
2024-01-24 14:03:42.012363: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [32,1]
	 [[{{node Placeholder/_1}}]]
2024-01-24 14:03:42.020883: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [32,1]
	

1998 [D loss: 0.592145, acc.: 60.94%] [G loss: 1.124064]
1/1 [==============================] - 0s 20ms/step
1999 [D loss: 0.528453, acc.: 71.88%] [G loss: 1.134566]
1/1 [==============================] - 0s 24ms/step
2000 [D loss: 0.528656, acc.: 75.00%] [G loss: 1.134740]
1/1 [==============================] - 0s 21ms/step


2024-01-24 14:03:42.248987: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [32,1]
	 [[{{node Placeholder/_1}}]]
2024-01-24 14:03:42.263241: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [32,1]
	 [[{{node Placeholder/_1}}]]
2024-01-24 14:03:42.273524: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [32,1]
	